In [1]:
#!pip uninstall tensorflow

In [2]:
#!pip install tensorflow-gpu==2.0.0

In [1]:
from tensorflow.compat.v1 import enable_eager_execution
import tensorflow as tf
import numpy as np
import os
import tensorflow_probability as tfp
from tensorflow.keras import layers
import time
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
from sklearn.metrics import roc_auc_score
import random
from sklearn.metrics import confusion_matrix
print(tf.__version__)

2.0.0


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
tf.compat.v1.Session(config=config)

## 參數設定

In [3]:
flatten_size = 32 * 32 * 3
img_size = 32
img_channel = 3
regular = 1e-5

z_dim = 128
h_dim = [32, 64, 128]
objective = 'deep-SVDD'     #'  'soft-boundary'


## 匯入資料（MNIST）

In [4]:
names = np.array(['airplane',
'automobile',
'bird',
'cat',
'deer',
'dog',
'frog',
'horse',
'ship',
'truck'])

In [5]:
location  = 'E:/咪聽/SVDD/deep SVDD/checkpoints/'
location  = 'D:/7107018014/checkpoint/'

In [6]:
def load_data(label_num, batch_size = 200, outlier_nu = 0.1):
    # 0: normal, 1: outlier
    
    x_train = np.load('D:/7107018014/cifar10/{}/{}_data{}_batch{}.npy'.format(label_num,'train',label_num, 0))
    y_train = np.load('D:/7107018014/cifar10/{}/{}_label{}_batch{}.npy'.format(label_num,'train',label_num, 0))
    x_train = np.transpose(x_train, (0, 2, 3, 1))
    
    
    for i in np.arange(2):
        data = np.load('D:/7107018014/cifar10/{}/{}_data{}_batch{}.npy'.format(label_num,'test',label_num, i))
        label = np.load('D:/7107018014/cifar10/{}/{}_label{}_batch{}.npy'.format(label_num,'test',label_num, i))
        
        data = np.transpose(data, (0, 2, 3, 1))
        try:
            x_test = np.vstack((x_test, data))
            y_test = np.hstack((y_test, label))
        except:
            x_test = data
            y_test = label
    ## outlier data
    for index in np.delete(np.arange(10), label_num):
        
        out_data = np.load('D:/7107018014/cifar10/{}/{}_data{}_batch{}.npy'.format(index,'train',index, 0))
        out_label = np.load('D:/7107018014/cifar10/{}/{}_label{}_batch{}.npy'.format(index,'train',index, 0))

        out_data = np.transpose(out_data, (0, 2, 3, 1))
        try:
            out_datas_ = np.vstack((out_datas_, out_data))
            out_labels_ = np.hstack((out_labels_, out_label))
        except:
            out_datas_ = out_data
            out_labels_ = out_label
        
    ### 製造outlier
    random.seed (1120)
    normal_n = len(x_train)
    outlier_n = len(out_datas_) ####
    indices = np.random.choice(outlier_n, int(normal_n * outlier_nu), replace=False)
    x_train = np.vstack( (x_train[y_train == label_num], out_datas_[indices]))
    y_train = np.hstack( (y_train[y_train == label_num], out_labels_[indices]))
    
    
    # Flatten the dataset
    x_train = x_train.reshape((-1, flatten_size))
    x_test = x_test.reshape((-1, flatten_size))
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')

    train_buf = x_train.shape[0]
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=train_buf)
    train_dataset = train_dataset.batch(batch_size)
    
    return train_dataset, x_train, x_test, y_train, y_test

## 建構模型

## Prtrain

In [7]:
class Autoencoder():
    
    def __init__(self, label_num, img_size = img_size, z_dim = z_dim, h_dim = h_dim, img_channel = img_channel):
        
        self.label_num = label_num
        self.flatten_size = img_size * img_size * img_channel
        self.z_dim = z_dim
        self.h_dim = h_dim
        self.encoder = self.make_encoder()
        self.decoder = self.make_decoder()
        self.ae_optimizer = tf.keras.optimizers.Adam(lr = 0.001, epsilon = 1e-6)
        

    def make_encoder(self):

        inputs = tf.keras.Input(shape=(self.flatten_size,), name='inputs')
        x = tf.reshape(inputs, shape = (-1, 32, 32, 3))
        x = tf.keras.layers.Conv2D(self.h_dim[0], (5, 5), activation='linear', padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization(epsilon = 1e-04)(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)

        x = tf.keras.layers.Conv2D(self.h_dim[1], (5, 5), activation='linear', padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization(epsilon = 1e-04)(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)
        
        x = tf.keras.layers.Conv2D(self.h_dim[2], (5, 5), activation='linear', padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization(epsilon = 1e-04)(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)

        x = tf.keras.layers.Flatten(name = 'Flatten')(x)
        x = tf.keras.layers.BatchNormalization(epsilon = 1e-04)(x)
        x = tf.keras.layers.Dense(self.z_dim, use_bias = False, name = 'dense')(x)
        
        
        model = tf.keras.Model(inputs=inputs, outputs=x)

        return model    
   
    def make_decoder(self):

        inputs = tf.keras.Input(shape=(self.z_dim,), name='inputs')
        x = tf.reshape(inputs, shape = (-1, 4, 4, 8))
        x = tf.keras.layers.LeakyReLU()(x)
        #x = tf.keras.layers.UpSampling2D((2, 2))(x)    

        x = tf.keras.layers.Conv2DTranspose(self.h_dim[2], (5, 5), padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)   

        x = tf.keras.layers.Conv2DTranspose(self.h_dim[1], (5, 5), padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        
        x = tf.keras.layers.Conv2DTranspose(self.h_dim[0], (5, 5), padding='same',  use_bias = False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ELU()(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x) 
        
        x = tf.keras.layers.Conv2DTranspose(3, (5, 5), activation='sigmoid', padding='same',  use_bias = False)(x)
        #x = tf.reshape(x, shape = (-1, self.flatten_size))
        x = tf.keras.layers.Flatten(name = 'Flatten')(x)
        model = tf.keras.Model(inputs=inputs, outputs=x)

        return model   
    
    @tf.function
    def train_ae_model(self, batch_x):
        with tf.GradientTape() as ae_tape:
            ae_output = self.decoder(self.encoder(batch_x, training=True), training=True)
            loss = tf.reduce_mean((ae_output - batch_x) ** 2)

        grads = ae_tape.gradient(loss, self.encoder.trainable_variables + self.decoder.trainable_variables)
        self.ae_optimizer.apply_gradients(zip(grads, self.encoder.trainable_variables + self.decoder.trainable_variables))

        return loss
    
    def train_ae( self, n_ae_epochs, train_dataset, ae_lr_milestone=250 ):
        #loss_list = []
        for epoch in range(n_ae_epochs):
            epoch_ae_loss_avg = tf.metrics.Mean()
            
            if (epoch+1) % ae_lr_milestone == 0:
                self.ae_optimizer.lr = 0.1 * self.ae_optimizer.lr

            for batch, (batch_x, batch_y) in enumerate(train_dataset):

                ae_loss = self.train_ae_model(batch_x)
                #epoch_ae_loss_avg(ae_loss)
            #loss_list.append(epoch_ae_loss_avg.result())
        
        self.encoder.save_weights('D:/7107018014/checkpoint/oneclassAE_label_{}' .format(self.label_num))
        #print('LOSS: {:.4f} '.format(epoch_ae_loss_avg.result()))
        #plt.title('label = {}'.format(self.label_num))
        #plt.plot(loss_list)
        #plt.show()
        

In [8]:
ae = Autoencoder('airplane')

In [9]:
encoder = ae.encoder
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 3072)]            0         
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        2400      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
elu (ELU)                    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        51200 

In [10]:
decoder = ae.decoder
decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 128)]             0         
_________________________________________________________________
tf_op_layer_Reshape_1 (Tenso [(None, 4, 4, 8)]         0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 4, 4, 128)         25600     
_________________________________________________________________
batch_normalization_4 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
elu_3 (ELU)                  (None, 4, 4, 128)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 8, 8, 128)         0   

## result 

In [11]:
def threshold_img(data, dists):
    dict_score = {}
    for i in range(len(dists)):
        dict_score[float(dists[i])] = i
    sort_score = [(k, dict_score[k]) for k in sorted(dict_score.keys())]
    
    n_digits = 20  # how many digits we will display
    fig = plt.figure(figsize=(20, 4))
    for i in range(1, n_digits):
        # display normal
        ax = plt.subplot(2, n_digits, i + 1)
        plt.imshow(data[sort_score[i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display anomaly
        ax = plt.subplot(2, n_digits, i + 1 + n_digits)
        plt.imshow(data[sort_score[-i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()


In [12]:
def threshold_uncertain_img(data, dists, r):
    dist2radius = (dists - r**2)
    normal = dist2radius[dist2radius<0]
    anomaly = dist2radius[dist2radius>0]
    normal_score = {}
    anomaly_score = {}
    for i in range(len(normal)):
        normal_score[float(normal[i])] = i
    for i in range(len(anomaly)):
        anomaly_score[float(anomaly[i])] = i
    normalsort_score = [(k, normal_score[k]) for k in sorted(normal_score.keys())]
    anomalysort_score = [(k, anomaly_score[k]) for k in sorted(anomaly_score.keys())]
    
    n_digits = min([len(normal),len(anomaly),20])#20  # how many digits we will display
    fig = plt.figure(figsize=(20, 4))
    #print('normal:', len(normal), 'anomaly', len(anomaly))
    for i in range(1, n_digits):
        # display normal
        
        ax = plt.subplot(2, n_digits, i + 1)
        plt.imshow(data[normalsort_score[-i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display anomaly
        ax = plt.subplot(2, n_digits, i + 1 + n_digits)
        plt.imshow(data[anomalysort_score[i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()

In [13]:
def distance_plot(label_num, dist, data, radius_end, objective, train = True, label = None,  nu = 0.1):
    treshold = radius_end ** 2
    if train:
        plt.title(str(objective) + 'label = {} train data'.format(label_num))
        if objective == 'soft-boundary':
            treshold = 0
            plt.hist(dis, rwidth=0.3, color='b' ,bins = 100)
            plt.axvline(treshold,  color='r')
        else:
            score = dist
            plt.hist(score, rwidth=0.3, color='b' ,bins = 100)
            plt.axvline(float(treshold),  color='g')
        plt.show()       
    
    else:
        plt.title(str(objective) + 'label = {} test data'.format(label_num))
        if objective == 'soft-boundary':
            plt.hist(dis[label!=label_num], rwidth=0.3, color='r' ,bins = 100,label = 'anomaly')
            plt.hist(dis[label==label_num], rwidth = 0.3, color = 'b',label = 'normal', bins = 100)
            plt.axvline(treshold,  color='g')
            plt.legend()
            plt.show()
            plt.hist(dist[label==label_num], rwidth=0.3, color='b' ,bins = 100,label = 'normal')
            plt.axvline(treshold, color='g')
            plt.legend()
            plt.show()
            plt.hist(dist[label!=label_num], rwidth=0.3, color='r' ,bins = 100,label = 'anomaly')
            plt.axvline(treshold, color='g')
        else:
            score = dist
            plt.hist(score[label!=label_num], rwidth=0.3, color='r' ,bins = 100,label = 'anomaly')
            plt.hist(score[label==label_num], rwidth=0.3, color='b' ,bins = 100,label = 'normal')
            plt.axvline(float(treshold), color='g')
            plt.legend()
            plt.show()
            plt.hist(score[label==label_num], rwidth=0.3, color='b' ,bins = 100,label = 'normal')
            plt.axvline(float(treshold), color='g')
            plt.legend()
            plt.show()
            plt.hist(score[label!=label_num], rwidth=0.3, color='r' ,bins = 100,label = 'anomaly')
            plt.axvline(float(treshold), color='g')
        plt.legend()
        plt.show()
    


In [14]:
def false_img(data, labels_,  dists, r, label_num):
    dist2radius = (dists - r**2)
    dist_n = dist2radius[labels_ != label_num]
    dist_p = dist2radius[labels_ == label_num]
    data_n = data[labels_ != label_num]
    data_p = data[labels_ == label_num]
    fn = dist_n[dist_n<0]
    fp = dist_p[dist_p>0]
    fn_score = {}
    fp_score = {}
    for i in range(len(fn)):
        fn_score[float(fn[i])] = i
    for i in range(len(fp)):
        fp_score[float(fp[i])] = i
        
    fn_score_sort = [(k, fn_score[k]) for k in sorted(fn_score.keys())]
    fp_score_sort = [(k, fp_score[k]) for k in sorted(fp_score.keys())]
    
    n_digits = min([len(fn),len(fp),20])#20  # how many digits we will display
    fig = plt.figure(figsize=(20, 4))
    #print('normal:', len(normal), 'anomaly', len(anomaly))
    for i in range(1, n_digits):
        # display fn
        ax = plt.subplot(2, n_digits, i + 1)
        plt.imshow(data_n[fn_score_sort[-i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display fp
        ax = plt.subplot(2, n_digits, i + 1 + n_digits)
        plt.imshow(data_p[fp_score_sort[i][1]].reshape(img_size, img_size, img_channel), cmap = plt.cm.gray)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()


In [15]:
class make_binary_model():
    def __init__(self, encoder, z_dim = z_dim):
        self.encoder = encoder
        self.z_dim = z_dim
        self.SVDD_model = self.make_model()
    
    
    def initializer(self):
        kernel_initializer = self.encoder.get_layer('dense').get_weights()[0]
        
        return kernel_initializer
    
    def make_model(self):
        
        kernel_initializer = self.initializer()
        
        self.encoder.trainable = True
        x = self.encoder.get_layer('Flatten').output
        x = tf.keras.layers.Dense(
        self.z_dim, activation = 'relu', kernel_initializer = tf.constant_initializer(kernel_initializer))(x)
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model_new = tf.keras.Model(inputs = self.encoder.input, outputs = outputs)
    
        return model_new
    
    

In [16]:
class train_classfication():
    
    def __init__(self, label_num , binary_model, train_dataset, nu ):
        names = np.array(['airplane',
        'automobile',
        'bird',
        'cat',
        'deer',
        'dog',
        'frog',
        'horse',
        'ship',
        'truck'])
        
        self.label_num = label_num
        self.label_name = names[label_num]
        self.nu = nu
        self.binary_model = binary_model
        self.label_num = label_num
        self.optimizer = tf.keras.optimizers.Adam(lr = 0.0001, epsilon = 1e-6)
        self.train_dataset = train_dataset
        
        
    def loss(self, batch_x, batch_y):
        
        avg_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=batch_y, logits=batch_x)
        end_loss = tf.reduce_mean(avg_loss)

        return end_loss

    @tf.function
    def train_model(self, batch_x, batch_y):
        with tf.GradientTape() as tape:
            output = self.binary_model(batch_x, training=True)
            output = tf.squeeze(output)
            output_loss = self.loss(output, batch_y)

        grads = tape.gradient(output_loss, self.binary_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.binary_model.trainable_variables))

        return output_loss
    
    def train( self, lr_milestone = 15):
        
        
        pre_loss = np.inf
        loss = 0.
        loss_list = []
        for epoch in range(60):
            epoch_ae_loss_avg = tf.metrics.Mean()
            
            if (epoch+1) % lr_milestone == 0:
                self.optimizer.lr = 0.1 * self.optimizer.lr

            for batch, (batch_x, batch_y) in enumerate(train_dataset):

                ae_loss = self.train_model(batch_x, batch_y)
                epoch_ae_loss_avg(ae_loss)
            loss_list.append(epoch_ae_loss_avg.result())
        
        self.binary_model.save_weights('D:/7107018014/checkpoint/binary_model_label_{}_outlier_{}' .format(
            self.label_name, self.nu))
        
        
    def score(self, data, label):
        self.binary_model.load_weights('D:/7107018014/checkpoint/binary_model_label_{}_outlier_{}' .format(
            self.label_name, self.nu))
        batch_size = 200
        train_dataset = tf.data.Dataset.from_tensor_slices((data, label))
        train_dataset = train_dataset.batch(batch_size)

        for batch, (batch_x,batch_y) in enumerate(train_dataset):
            output = self.binary_model(batch_x, training=False)
            
            if batch ==0:
                modeloutput = output
                labels_ = batch_y
                datas = batch_x
            else:
                modeloutput = tf.concat([modeloutput, output], 0)
                labels_ = tf.concat([labels_, batch_y], 0)
                datas = tf.concat([datas, batch_x], 0)
                
        labels_ = labels_.numpy()
        datas = datas.numpy()
        
        return modeloutput, labels_, datas
    
    

    
    def result(self, scores, y_test, show_img = False):
        
        y_true = y_test==self.label_num
        test_auc = roc_auc_score(y_true, scores)
        if show_img:
            print('AUC:', test_auc)
            
        return test_auc 
    
    def test_result(self, x_test, y_test, show_img = False, load_model = False, search_u = False):
        
        scores, labels_, datas = self.score(x_test, y_test)
        test_auc = self.result(scores, labels_, show_img = show_img)
        if show_img:
            distance_plot(self.label_num, dist, self.objective, name =  self.label_num,
                          train = False, label = y_test, nu = self.nu )
            print('test label = {} uncertain_img'.format(self.label_num))
            threshold_uncertain_img(x_test, dist, radius_end)
            print('test label = {} distance oder'.format(self.label_num))
            threshold_img(x_test, dist)
            
        if search_u:
            print('u = {}, auc = {}'.format(self.nu, test_auc))
        return test_auc
    

In [25]:
nu = 0.0
outlier_nu = 0.
lr_milestone = 50
result = np.zeros((10, 4))

for label_num in np.arange(10):
    print("Loading ",names[label_num])
    train_dataset, x_train, x_test, y_train, y_test = load_data(label_num = label_num, outlier_nu = outlier_nu)
    pre_time = time.time()
    ae = Autoencoder(label_num)
    ae.train_ae( 50, train_dataset)
    encoder = ae.encoder
    make_binary = make_binary_model(encoder)
    binary_model = make_binary.make_model() 
    classfication = train_classfication(label_num , binary_model, train_dataset, nu )
    classfication.train(lr_milestone)
    test_auc = classfication.test_result(x_test, y_test, load_model = True)
    train_time = time.time() - pre_time
    print('auc = ', test_auc)
    result[label_num, 0] = label_num
    result[label_num, 2] = test_auc


Loading  airplane
auc =  0.5116491111111111
Loading  automobile
auc =  0.5624785555555556
Loading  bird
auc =  0.5000555555555555
Loading  cat
auc =  0.5
Loading  deer
auc =  0.47885738888888896
Loading  dog
auc =  0.5003335555555556
Loading  frog
auc =  0.5028213333333332
Loading  horse
auc =  0.48058877777777775
Loading  ship
auc =  0.5182690555555555
Loading  truck
auc =  0.5126814444444444


In [26]:
result

array([[0.        , 0.        , 0.51164911, 0.        ],
       [1.        , 0.        , 0.56247856, 0.        ],
       [2.        , 0.        , 0.50005556, 0.        ],
       [3.        , 0.        , 0.5       , 0.        ],
       [4.        , 0.        , 0.47885739, 0.        ],
       [5.        , 0.        , 0.50033356, 0.        ],
       [6.        , 0.        , 0.50282133, 0.        ],
       [7.        , 0.        , 0.48058878, 0.        ],
       [8.        , 0.        , 0.51826906, 0.        ],
       [9.        , 0.        , 0.51268144, 0.        ]])

In [27]:
nu = 0.1
outlier_nu = 0.1
lr_milestone = 50
result = np.zeros((10, 4))

for label_num in np.arange(10):
    print("Loading ",names[label_num])
    train_dataset, x_train, x_test, y_train, y_test = load_data(label_num = label_num, outlier_nu = outlier_nu)
    pre_time = time.time()
    ae = Autoencoder(label_num)
    ae.train_ae( 350, train_dataset)
    encoder = ae.encoder
    make_binary = make_binary_model(encoder)
    binary_model = make_binary.make_model() 
    classfication = train_classfication(label_num , binary_model, train_dataset, nu )
    classfication.train(lr_milestone)
    test_auc = classfication.test_result(x_test, y_test, load_model = True)
    train_time = time.time() - pre_time
    print('auc = ', test_auc)
    result[label_num, 0] = label_num
    result[label_num, 2] = test_auc


Loading  airplane
auc =  0.5802643888888889
Loading  automobile
auc =  0.499404
Loading  bird
auc =  0.5
Loading  cat
auc =  0.5000555555555555
Loading  deer
auc =  0.4985548333333333
Loading  dog
auc =  0.5
Loading  frog
auc =  0.4146121666666666
Loading  horse
auc =  0.5
Loading  ship
auc =  0.49727655555555555
Loading  truck
auc =  0.500278


In [17]:
nu = 0.2
outlier_nu = 0.2
lr_milestone = 50
result = np.zeros((10, 4))

for label_num in np.arange(10):
    print("Loading ",names[label_num])
    train_dataset, x_train, x_test, y_train, y_test = load_data(label_num = label_num, outlier_nu = outlier_nu)
    pre_time = time.time()
    ae = Autoencoder(label_num)
    ae.train_ae( 350, train_dataset)
    encoder = ae.encoder
    make_binary = make_binary_model(encoder)
    binary_model = make_binary.make_model() 
    classfication = train_classfication(label_num , binary_model, train_dataset, nu )
    classfication.train(lr_milestone)
    test_auc = classfication.test_result(x_test, y_test, load_model = True)
    train_time = time.time() - pre_time
    print('auc = ', test_auc)
    result[label_num, 0] = label_num
    result[label_num, 2] = test_auc


Loading  airplane
auc =  0.4030138333333334
Loading  automobile
auc =  0.5010073888888888
Loading  bird
auc =  0.4864466666666667
Loading  cat
auc =  0.509422111111111
Loading  deer
auc =  0.5
Loading  dog
auc =  0.5
Loading  frog
auc =  0.49997394444444443
Loading  horse
auc =  0.5052738888888889
Loading  ship
auc =  0.4962263333333333
Loading  truck
auc =  0.49036027777777774


In [18]:
nu = 0.3
outlier_nu = 0.3
lr_milestone = 50
result = np.zeros((10, 4))

for label_num in np.arange(10):
    print("Loading ",names[label_num])
    train_dataset, x_train, x_test, y_train, y_test = load_data(label_num = label_num, outlier_nu = outlier_nu)
    pre_time = time.time()
    ae = Autoencoder(label_num)
    ae.train_ae( 350, train_dataset)
    encoder = ae.encoder
    make_binary = make_binary_model(encoder)
    binary_model = make_binary.make_model() 
    classfication = train_classfication(label_num , binary_model, train_dataset, nu )
    classfication.train(lr_milestone)
    test_auc = classfication.test_result(x_test, y_test, load_model = True)
    train_time = time.time() - pre_time
    print('auc = ', test_auc)
    result[label_num, 0] = label_num
    result[label_num, 2] = test_auc


Loading  airplane
auc =  0.47572288888888886
Loading  automobile
auc =  0.5058556666666667
Loading  bird
auc =  0.5
Loading  cat
auc =  0.500391
Loading  deer
auc =  0.5
Loading  dog
auc =  0.5
Loading  frog
auc =  0.5367641111111112
Loading  horse
auc =  0.5006681666666667
Loading  ship
auc =  0.46861366666666665
Loading  truck
auc =  0.6304242222222222
